# TP2 : Modélisation de la Sécheresse Agricole - Nord Côte d'Ivoire 2016
## Formation CLIMADA - Direction Générale de l'Économie (DGE)

### Objectifs du TP
1. **Calculer** les indices SPI (Standardized Precipitation Index) multi-stations
2. **Spatialiser** la sécheresse par interpolation krigeage
3. **Modéliser** l'exposition agricole multi-cultures (coton, riz, maïs)
4. **Évaluer** les impacts économiques sectoriels
5. **Analyser** des scénarios d'adaptation

### Contexte
La saison agricole 2016 a été marquée par une sécheresse sévère dans le Nord de la Côte d'Ivoire, affectant particulièrement les cultures de coton, riz pluvial et maïs. Ce TP vise à quantifier ces impacts avec CLIMADA.

### Durée estimée : 3 heures

---

## Module 1 : Données Agro-Météorologiques et Calcul SPI (45 min)

### 1.1 Import des bibliothèques et configuration

In [ ]:
# Bibliothèques de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Bibliothèques géospatiales
import geopandas as gpd
import rasterio as rio
from shapely.geometry import Point, Polygon
import folium

# Bibliothèques statistiques
from scipy import stats
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern
from sklearn.metrics import mean_squared_error

# CLIMADA core
from climada.hazard import Hazard
from climada.entity import Exposures, ImpactFunc, ImpactFuncSet
from climada.engine import Impact
from climada.util.coordinates import get_grid_points

# Configuration affichage
plt.style.use('seaborn-v0_8')
sns.set_palette("RdYlBu_r")  # Palette adaptée à la sécheresse
%matplotlib inline

print("✅ Toutes les bibliothèques importées avec succès!")
print(f"Version CLIMADA: {climada.__version__}")

### 1.2 Chargement des données météorologiques multi-stations

In [ ]:
# Chargement données sécheresse 2016 - Nord Côte d'Ivoire
# Note: En formation réelle, ces données proviennent de SODEXAM
drought_data = pd.read_csv('data/secheresse_agriculture_2016.csv')

# Conversion colonne datetime
drought_data['date'] = pd.to_datetime(drought_data['date'])

# Informations sur les stations
stations_info = drought_data.groupby('station').agg({
    'latitude': 'first',
    'longitude': 'first',
    'altitude': 'first',
    'precip_mm': 'count'
}).rename(columns={'precip_mm': 'nb_observations'})

print("🌧️ DONNÉES MÉTÉOROLOGIQUES MULTI-STATIONS - NORD CI:")
print(f"Période: {drought_data['date'].min()} à {drought_data['date'].max()}")
print(f"Nombre de stations: {drought_data['station'].nunique()}")
print(f"Nombre total d'observations: {len(drought_data):,}")

print("\n📍 INFORMATIONS STATIONS:")
print("="*80)
print(f"{'Station':<15} {'Latitude':<10} {'Longitude':<11} {'Altitude':<9} {'Obs.':<6}")
print("="*80)
for station, info in stations_info.iterrows():
    print(f"{station:<15} {info['latitude']:<10.3f} {info['longitude']:<11.3f} {info['altitude']:<9.0f} {info['nb_observations']:<6}")

# Vérification complétude des données
print("\n📊 COMPLÉTUDE DES DONNÉES PAR STATION:")
for station in drought_data['station'].unique():
    station_data = drought_data[drought_data['station'] == station]
    total_months = len(station_data)
    missing_data = station_data['precip_mm'].isna().sum()
    completeness = ((total_months - missing_data) / total_months) * 100
    print(f"  {station}: {completeness:.1f}% complète ({missing_data} valeurs manquantes)")

# Aperçu des données
display(drought_data.head(10))

### 1.3 Calcul des indices SPI (Standardized Precipitation Index)

In [ ]:
def calculate_spi(precip_series, scale=3, distribution='gamma'):
    """
    Calcule l'indice SPI (Standardized Precipitation Index)
    
    Parameters:
    -----------
    precip_series : pandas.Series
        Série temporelle des précipitations mensuelles
    scale : int
        Échelle temporelle en mois (1, 3, 6, 12)
    distribution : str
        Distribution statistique ('gamma' ou 'normal')
        
    Returns:
    --------
    spi_values : pandas.Series
        Valeurs SPI correspondantes
    """
    # Calcul des cumuls glissants
    if scale > 1:
        rolling_precip = precip_series.rolling(window=scale, min_periods=scale).sum()
    else:
        rolling_precip = precip_series.copy()
    
    # Suppression des valeurs manquantes
    valid_data = rolling_precip.dropna()
    
    if len(valid_data) < 30:  # Minimum 30 valeurs pour ajustement
        return pd.Series(index=precip_series.index, dtype=float)
    
    # Ajustement de la distribution
    if distribution == 'gamma':
        # Ajustement loi gamma (plus appropriée pour précipitations)
        params = stats.gamma.fit(valid_data, floc=0)
        cdf_values = stats.gamma.cdf(rolling_precip, *params)
    else:
        # Ajustement loi normale
        params = stats.norm.fit(valid_data)
        cdf_values = stats.norm.cdf(rolling_precip, *params)
    
    # Transformation en SPI (quantiles de la loi normale standard)
    # Éviter les valeurs extrêmes (0 et 1)
    cdf_values = np.clip(cdf_values, 0.001, 0.999)
    spi_values = stats.norm.ppf(cdf_values)
    
    return pd.Series(spi_values, index=rolling_precip.index)

# Calcul SPI-3 pour toutes les stations
print("⚙️ Calcul des indices SPI-3 en cours...")

spi_results = []
for station in drought_data['station'].unique():
    station_data = drought_data[drought_data['station'] == station].copy()
    station_data = station_data.sort_values('date')
    
    # Calcul SPI-3
    spi_3 = calculate_spi(station_data['precip_mm'], scale=3)
    
    # Ajout des résultats
    station_data['spi_3'] = spi_3
    spi_results.append(station_data)

# Consolidation des résultats
drought_data_spi = pd.concat(spi_results, ignore_index=True)

print("✅ Calcul SPI terminé!")

# Classification des niveaux de sécheresse
def classify_drought(spi_value):
    """Classifie le niveau de sécheresse selon la valeur SPI"""
    if pd.isna(spi_value):
        return 'Données manquantes'
    elif spi_value >= 0:
        return 'Normal à humide'
    elif spi_value >= -1:
        return 'Sécheresse légère'
    elif spi_value >= -1.5:
        return 'Sécheresse modérée'
    elif spi_value >= -2:
        return 'Sécheresse sévère'
    else:
        return 'Sécheresse extrême'

drought_data_spi['drought_category'] = drought_data_spi['spi_3'].apply(classify_drought)

# Statistiques SPI par station
print("\n📊 STATISTIQUES SPI-3 PAR STATION (2016):")
spi_stats = drought_data_spi[drought_data_spi['date'].dt.year == 2016].groupby('station')['spi_3'].agg([
    'min', 'mean', 'max', 'std'
]).round(2)

print("="*70)
print(f"{'Station':<15} {'Min':<8} {'Moyenne':<10} {'Max':<8} {'Écart-type':<10}")
print("="*70)
for station, stats_row in spi_stats.iterrows():
    print(f"{station:<15} {stats_row['min']:<8} {stats_row['mean']:<10} {stats_row['max']:<8} {stats_row['std']:<10}")

# Identification des mois les plus secs en 2016
drought_2016 = drought_data_spi[drought_data_spi['date'].dt.year == 2016]
worst_drought = drought_2016.loc[drought_2016['spi_3'].idxmin()]

print(f"\n🔥 SÉCHERESSE LA PLUS SÉVÈRE EN 2016:")
print(f"Station: {worst_drought['station']}")
print(f"Date: {worst_drought['date'].strftime('%B %Y')}")
print(f"SPI-3: {worst_drought['spi_3']:.2f} ({classify_drought(worst_drought['spi_3'])})")

### 1.4 Visualisation des séries temporelles SPI

In [ ]:
# Graphiques des séries temporelles SPI
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

stations = drought_data_spi['station'].unique()
colors = ['blue', 'green', 'red', 'orange']

for i, station in enumerate(stations):
    station_data = drought_data_spi[drought_data_spi['station'] == station]
    
    ax = axes[i]
    
    # Ligne SPI
    ax.plot(station_data['date'], station_data['spi_3'], 
           color=colors[i], linewidth=2, label=f'SPI-3 {station}')
    
    # Zones colorées selon niveau de sécheresse
    ax.axhspan(-1, 0, alpha=0.2, color='yellow', label='Sécheresse légère')
    ax.axhspan(-1.5, -1, alpha=0.3, color='orange', label='Sécheresse modérée')
    ax.axhspan(-2, -1.5, alpha=0.4, color='red', label='Sécheresse sévère')
    ax.axhspan(-3, -2, alpha=0.5, color='darkred', label='Sécheresse extrême')
    
    # Ligne de référence
    ax.axhline(0, color='black', linestyle='--', alpha=0.5)
    
    # Mise en évidence de 2016
    data_2016 = station_data[station_data['date'].dt.year == 2016]
    ax.scatter(data_2016['date'], data_2016['spi_3'], 
              color='red', s=50, zorder=5, alpha=0.8)
    
    ax.set_title(f'Évolution SPI-3 - {station}', fontweight='bold')
    ax.set_ylabel('SPI-3')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-3, 3)
    
    if i >= 2:  # Axes du bas
        ax.set_xlabel('Date')
    
    # Légende seulement pour le premier graphique
    if i == 0:
        ax.legend(loc='upper right', fontsize=8)

plt.suptitle('Évolution des Indices SPI-3 - Nord Côte d\'Ivoire\n(Points rouges = Année 2016)', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Graphique comparatif 2016
plt.figure(figsize=(14, 8))

for i, station in enumerate(stations):
    station_2016 = drought_data_spi[
        (drought_data_spi['station'] == station) & 
        (drought_data_spi['date'].dt.year == 2016)
    ]
    
    plt.plot(station_2016['date'], station_2016['spi_3'], 
            marker='o', linewidth=3, markersize=8, 
            label=station, color=colors[i])

# Zones de sécheresse
plt.axhspan(-1, 0, alpha=0.2, color='yellow', label='Sécheresse légère')
plt.axhspan(-1.5, -1, alpha=0.3, color='orange', label='Sécheresse modérée')
plt.axhspan(-2, -1.5, alpha=0.4, color='red', label='Sécheresse sévère')
plt.axhspan(-3, -2, alpha=0.5, color='darkred', label='Sécheresse extrême')

plt.axhline(0, color='black', linestyle='--', alpha=0.7)
plt.xlabel('Mois 2016', fontsize=12)
plt.ylabel('SPI-3', fontsize=12)
plt.title('Comparaison SPI-3 par Station - Saison Agricole 2016\n(Mai à Octobre)', 
          fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.ylim(-2.5, 1)
plt.tight_layout()
plt.show()

print("📈 Graphiques SPI générés avec succès!")